In [49]:
import pandas as pd
import numpy as np
import os
from numpy import genfromtxt

glycos = pd.read_csv('FILE1') #Glycosylated file from ring
Unglycos = pd.read_csv('FILE2') #Unglycosylated file from ring
glycos = glycos.rename(columns={'frequency': 'frequency_gly'})
Unglycos = Unglycos.rename(columns={'frequency': 'frequency_ungly'})

###### THIS SECTION CAN BE IMPLEMETNED TO DETERMINE THE NUMBER OF RESIDIES TO INCLUDE BETWEEN INTERACTION I.E. i+4 is can be used to exclude all interaction within 4 #####

#merged_2 = pd.merge(glycos, Unglycos, on= ['source', 'target', 'interaction', 'atom1', 'atom2', 'energy', 'distance' ], how='outer')
#merged_2 = merged_2.fillna(0)
#merged_2['freq_diff'] = merged_2['frequency_gly'].sub(merged_2['frequency_ungly'])
#merged_2['source_number'] = merged_2['source']
#merged_2['source_number'] = merged_2.source_number.str.extract('(\d+)')
#merged_2['target_number'] = merged_2['target']
#merged_2['target_number'] = merged_2.target_number.str.extract('(\d+)')
#merged_2['target_number'] = merged_2['target_number'].astype(int)
#merged_2['source_number'] = merged_2['source_number'].astype(int)
#merged_2['residue_difference'] = merged_2['source_number'] - merged_2['target_number']

#merged_greater_than = merged_2[merged_2.residue_difference > 0]
#merged_less_than = merged_2[merged_2.residue_difference < 0]
#four_resi = (merged_greater_than, merged_less_than)
#merged_four_residues = pd.concat(four_resi)

glycos['extracted_source'] = glycos.source.str.extract('(\d+)')
glycos['extracted_target'] = glycos.target.str.extract('(\d+)')
glycos['extracted_source'] = glycos['extracted_source'].astype(int)
glycos['extracted_target'] = glycos['extracted_target'].astype(int)
glycos['uniqueID'] = ((glycos['extracted_source'] * 3.153)*(glycos['extracted_target'] * 3.153))

Unglycos['extracted_source'] = Unglycos.source.str.extract('(\d+)')
Unglycos['extracted_target'] = Unglycos.target.str.extract('(\d+)')
Unglycos['extracted_source'] = Unglycos['extracted_source'].astype(int)
Unglycos['extracted_target'] = Unglycos['extracted_target'].astype(int)
Unglycos['uniqueID'] = ((Unglycos['extracted_source'] * 3.153)*(Unglycos['extracted_target'] * 3.153))

merged3 = pd.merge(glycos, Unglycos, on= ['extracted_source', 'extracted_target', 'interaction', 'atom1', 'atom2'], how='inner')
merged3['freq_diff'] = merged3['frequency_ungly'] - (merged3['frequency_gly'])

merged3_less_than_minues_10 = merged3[merged3.freq_diff < -0.1] #remove frequncies below -10% change
merged3_less_than_plus_10 = merged3[merged3.freq_diff > 0.1] #remove frequncies below +10% change
greater_than_10_diff = (merged3_less_than_minues_10,merged3_less_than_plus_10)

Nodes_and_edges_greater_than_10_percent_difference_in_frequency = pd.concat(greater_than_10_diff)
Nodes_and_edges_greater_than_10_percent_difference_in_frequency.to_csv('/content/concatenated_ring_results.csv')